In [1]:
using DataFrames, JLD, LowRankModels, PyPlot, MLBase, CSV
include("proxgrad.jl")
using LowRankModels

INFO: Precompiling module CSV.


In [2]:
#lets heat up this laptop
BLAS.set_num_threads(8)

In [68]:
function shuffle(X)
    n = size(X)[1]
    rand_ind = sortperm(rand(n))
    X_s = X[rand_ind,:]
    return X_s
end

MASTER = load("MASTER.jld")["MASTER"]
MASTER = shuffle(MASTER)

,ORTC,ORSC,TCOR,TCSH,CNSH,MCSH,AWSH,f0t9,f10t29,f30t59,f60t299,ft30m,ARSP,AESP,PISH,PIAM,PIAT,AQSH,AQAT,OQSH,OQAM,OQAT,TICKER,MONTH,ID,PRC,SHROUT,VOL,MMCNT,RVOL,AVG_VIX,PS_LEVEL,PS_INNOV,PS_VWF
1,12,22,149,126520,3000,52126,0,52126,0,0,0,0,0.0036,0.0227,1812,0.01,0.0,48314,0.0,2000,0.015,0.0,RPV,1,RPV1,52.627995,21839.5,305202.65,NaN,0.20701299999999997,19.1195,8.25e-5,0.032193429,-0.01201946
2,12,22,219,183790,710,158759,300,159059,0,0,0,0,0.0015,0.0116,22088,0.0073,0.0,134171,0.0,2800,0.0136,0.0,IDV,6,IDV6,33.692727272727275,129793.18181818182,718759.5,NaN,0.15316090909090907,14.33954545,0.000675371,0.014720609,0.014262069
3,12,23,32,93797,2000,54161,28475,82636,0,0,0,0,0.022,0.015,16243,0.0138,0.0,63393,0.0,3000,0.01,0.0,GENE,5,GENE5,4.295,8744.0,822572.65,NaN,0.7371988999999999,13.3385,0.016801706,0.016479423,0.04530087
4,12,21,1111,143530,500,124565,4927,129492,0,0,0,0,0.0239,0.0572,21744,0.0254,0.0,105198,0.0,2550,0.0135,0.0,HABT,2,HABT2,31.6021052631579,8975.0,293881.84210526315,NaN,NaN,15.90421053,-0.014578715,0.001904306,-0.01277631
5,11,24,7,39000,0,8400,30600,39000,0,0,0,0,-0.0011,0.0045,12050,0.01,0.0,25550,0.0,1400,0.01,0.0,SYMX,4,SYMX4,1.1812095238095235,76157.14285714286,1.3295189523809524e6,NaN,0.9118905714285714,13.4947619,-0.002403011,0.02698389,-0.012508165
6,12,21,331,44137,1795,39056,910,39966,0,0,0,0,0.0088,0.0215,1601,0.0159,0.0,37919,0.0,446,0.0161,0.0,AMBR,3,AMBR3,8.985000000000001,25777.909090909092,62557.5,NaN,0.5051987727272728,14.81272727,-0.041376827,-0.014525278,0.00912096
7,12,21,3832,500330,23812,349265,4645,353910,0,0,0,0,0.0344,0.0203,61770,0.0101,0.0,283659,0.0,8481,0.0149,0.0,TIF,2,TIF2,88.73526315789474,129354.0,1.1228615789473683e6,NaN,0.3743193157894737,15.90421053,-0.014578715,0.001904306,-0.01277631
8,12,22,59,40881,1594,18244,7823,26067,0,0,0,0,0.0083,0.0182,8113,0.0211,0.0,17018,0.1,936,0.01,0.0,AOS,1,AOS1,56.10699999999999,76364.85,640795.5,NaN,0.21468564999999998,19.1195,8.25e-5,0.032193429,-0.01201946
9,11,21,3,300,0,200,100,300,0,0,0,0,-0.21,0.14,0,0.0,0.0,300,0.0,0,0.0,0.0,WRLD,4,WRLD4,75.51642857142858,8970.0,251186.61904761905,NaN,0.5998951428571428,13.4947619,-0.002403011,0.02698389,-0.012508165
10,11,21,84,9942,0,6270,3672,9942,0,0,0,0,-0.0179,0.0132,3531,0.0196,0.0,4711,0.0,1700,0.0121,0.0,CFX,7,CFX7,42.178636363636365,123850.86363636363,1.3480205e6,NaN,0.16289254545454546,14.35272727,0.023122246,0.025768641,0.016226211


In [69]:
MASTER = MASTER[MASTER[:TCOR] .>=0,:];
size(MASTER)

(167379, 34)

In [63]:
Xdf = DataFrame()

Xdf[:VIX] = MASTER[:AVG_VIX]
Xdf[:PSL] = MASTER[:PS_LEVEL]
Xdf[:PSIN] = MASTER[:PS_INNOV]
Xdf[:PSV] = MASTER[:PS_VWF]
Xdf[:MK] = Int.(MASTER[:ORTC] .== 11)
Xdf[:MKL] = Int.(MASTER[:ORTC] .== 12)
Xdf[:SHPO] = MASTER[:TCSH]./MASTER[:TCOR]
Xdf[:PVOL] = MASTER[:VOL]./MASTER[:SHROUT]
Xdf[:DVOL] = MASTER[:VOL].*MASTER[:PRC]
Xdf[:MCAP] = MASTER[:SHROUT].*MASTER[:PRC]
Xdf[:RVOLA] = MASTER[:RVOL]
Xdf[:PRC] = MASTER[:PRC]
Xdf[:AWSHP] = MASTER[:AWSH]./(MASTER[:TCSH] .- MASTER[:CNSH])
Xdf[:SLOW] = (MASTER[:f10t29].+MASTER[:f30t59].+MASTER[:f60t299].+MASTER[:ft30m])./MASTER[:TCSH]

#this is Y, will remove
Xdf[:AESP] = MASTER[:AESP]

size(Xdf)


(52843, 15)

In [64]:
fields = names(Xdf)
for f in fields
    Xdf[f][isnan.(Xdf[f])] = NA
end
completecases!(Xdf);
@show size(Xdf)
@show sum(Xdf[:AESP].>=0)
@show mean(Xdf[:AESP])

size(Xdf) = (52017, 15)
sum(Xdf[:AESP] .>= 0) = 51714
mean(Xdf[:AESP]) = 0.026451535075071614


0.026451535075071614

In [7]:
#PyPlot.plt[:hist](Xdf[:AESP][Xdf[:AESP].<=.05,:],300)
#xlim([0,.05])

In [65]:

size(Xdf)

sd = names(Xdf)

#dont standardize the one-hot encodings
sd = sd[sd.!=:MLIM]
sd = sd[sd.!=:LIM]

@show std_Y = std(Xdf[:AESP])
mean_Y = mean(Xdf[:AESP])
Y_true = Xdf[:AESP]

#standardize everything else
function standard(x)
    x_s = x .- mean(x)
    x_s = x_s ./ std(x)
    return x_s
end


#OLS Scaling Invariant, Ignore for now

for s in sd
    Xdf[s] = standard(Xdf[s])
end


Y = Xdf[:AESP]
delete!(Xdf,:AESP)

X = Array{Float64}(Xdf)

#offset
X = [ones(size(X)[1],1) X]

Y = Vector(Y)

fields = [:OFFSET; fields]


std_Y = std(Xdf[:AESP]) = 0.0696288358925823


16-element Array{Symbol,1}:
 :OFFSET
 :VIX   
 :PSL   
 :PSIN  
 :PSV   
 :MK    
 :MKL   
 :SHPO  
 :PVOL  
 :DVOL  
 :MCAP  
 :RVOLA 
 :PRC   
 :AWSHP 
 :SLOW  
 :AESP  

In [9]:
n = size(X)[1]


w_basic = X\Y


Yhat = X*w_basic

@show mean_absolute_error = sum(abs(Yhat - Y))/n
@show mse = sum((Yhat - Y).^2)/n

yClass = Int.(Y .> 0)


predClass = Yhat .> 0;

@show correct_classifications = sum(predClass .== yClass)/n


@show mean(Y)

@show sum(Y .> 0)/n

@show sum(Y.==0)
@show sum(Y.<0)/n
@show std(Y)
@show w_basic
@show fields


mean_absolute_error = sum(abs(Yhat - Y)) / n = 0.24839256363008966
mse = sum((Yhat - Y) .^ 2) / n = 0.8653346967567861
correct_classifications = sum(predClass .== yClass) / n = 0.7270932551797126
mean(Y) = -3.991325661859153e-17
sum(Y .> 0) / n = 0.25594293157487685
sum(Y .== 0) = 0
sum(Y .< 0) / n = 0.7440570684251231
std(Y) = 1.0
w_basic = [-2.35889e-16, 0.0169864, -0.0312804, 0.0560984, 0.0195524, -0.0268317, 0.0268317, -0.0455121, -0.0296663, -0.0345981, -0.0885337, 0.0888095, 0.385833, -0.00922357, 0.0435721]
fields = Symbol[:OFFSET, :VIX, :PSL, :PSIN, :PSV, :MK, :MKL, :SHPO, :PVOL, :DVOL, :MCAP, :RVOLA, :PRC, :AWSHP, :SLOW, :AESP]


16-element Array{Symbol,1}:
 :OFFSET
 :VIX   
 :PSL   
 :PSIN  
 :PSV   
 :MK    
 :MKL   
 :SHPO  
 :PVOL  
 :DVOL  
 :MCAP  
 :RVOLA 
 :PRC   
 :AWSHP 
 :SLOW  
 :AESP  

In [10]:
Y_class2 = Int.(Y.>median(Y))
pred_class2 = Int.(Yhat.>median(Y))

sum(Y_class2 .== pred_class2)./n

0.6597569910856026

In [11]:

function train_test(X,Y,p)
    n = size(X)[1]
    ntrain = Int(floor(p*n))
    X_train = X[1:ntrain,:]
    Y_train = Y[1:ntrain,:]
    X_test = X[(ntrain+1):n,:]
    Y_test = Y[(ntrain+1):n,:]
    return X_train, Y_train, X_test, Y_test

end

X_train, Y_train, X_test, Y_test = train_test(X,Y,0.8);

In [12]:
w_basic = X_train\Y_train


Yhat_train = X_train*w_basic

Yhat = X_test*w_basic
ntest = size(Y_test)[1]
ntrain = size(Y_train)[1]


@show mean_absolute_error_train = sum(abs(Yhat_train - Y_train))/ntrain
@show mse_train = sum((Yhat_train - Y_train).^2)/ntrain
@show mean_absolute_error_test = sum(abs(Yhat - Y_test))/ntest
@show mse_test = sum((Yhat - Y_test).^2)/ntest


mean_absolute_error_train = sum(abs(Yhat_train - Y_train)) / ntrain = 0.24869960211444642
mse_train = sum((Yhat_train - Y_train) .^ 2) / ntrain = 0.9150186097153072
mean_absolute_error_test = sum(abs(Yhat - Y_test)) / ntest = 0.24999290861524143
mse_test = sum((Yhat - Y_test) .^ 2) / ntest = 0.6705984071923042


0.6705984071923042

In [13]:
k = 10
all_inds = Array(Array[1:ntrain][1])
cv_inds = collect(Kfold(ntrain, k)); 

ml1_huber = zeros(k,1)
ml1_ls = zeros(k,1)
mse_huber = zeros(k,1)
mse_ls = zeros(k,1)


loss = L1Loss()
reg = ZeroReg()

for i = 1:k
    cvtest = setdiff(all_inds,cv_inds[i])
    X_cv = X_train[cv_inds[i],:]
    Y_cv = Y_train[cv_inds[i],:]
    X_cvt = X_train[cvtest,:]
    Y_cvt = Y_train[cvtest,:]
    
    ntest = size(Y_cvt)[1];
    
    
    
    w_ls = X_cv\Y_cv 
    yhat_ls = X_cvt*w_ls
    
    w_hub = proxgrad(loss,reg,X_cv,Y_cv[:])
    yhat_hub = X_cvt*w_hub
    
    
    ml1_ls[i] = float(sum(abs(yhat_ls - Y_cvt))/ntest)
    ml1_huber[i] = float(sum(abs(yhat_hub - Y_cvt))/ntest)
    mse_ls[i] = sum((yhat_ls - Y_cvt).^2)/ntest
    mse_huber[i] = sum((yhat_hub - Y_cvt).^2)/ntest
    
end



In [14]:
@show mean(ml1_ls);
@show mean(ml1_huber);

@show mean(mse_ls);
@show mean(mse_huber);

mean(ml1_ls) = 0.24889575737031472
mean(ml1_huber) = 0.215805162312531
mean(mse_ls) = 0.921524324419182
mean(mse_huber) = 0.9641691732062867


In [15]:
all_inds = Array(Array[1:ntrain][1])
cv_inds = collect(Kfold(ntrain, k)); 
L1 = zeros(100)
w_arr = []
for ks = 1:15
    reg = KSparseConstraint(ks)
    mL1 = zeros(15,1)
    for i = 1:k
        cvtest = setdiff(all_inds,cv_inds[i])
        X_cv = X_train[cv_inds[i],:]
        Y_cv = Y_train[cv_inds[i],:]
        X_cvt = X_train[cvtest,:]
        Y_cvt = Y_train[cvtest,:]
    
        ntest = size(Y_cvt)[1];
    
    
    
        w_ls = X_cv\Y_cv 
        yhat_ls = X_cvt*w_ls
    
        w_hub = proxgrad(loss,reg,X_cv,Y_cv[:])
        yhat_hub = X_cvt*w_hub
    
    
        ml1_ls[i] = float(sum(abs(yhat_ls - Y_cvt))/ntest)
        ml1_huber[i] = float(sum(abs(yhat_hub - Y_cvt))/ntest)
        mse_ls[i] = sum((yhat_ls - Y_cvt).^2)/ntest
        mse_huber[i] = sum((yhat_hub - Y_cvt).^2)/ntest
        
        push!(w_arr,w_hub)
    
    end

    L1[ks] = mean(ml1_huber)
    @show L1[ks]
    
end
    



L1[ks] = 0.2497508914859426
L1[ks] = 0.23197598420388427
L1[ks] = 0.22781505632727952
L1[ks] = 0.2241192353747369
L1[ks] = 0.21845384250261043
L1[ks] = 0.2168627271627833
L1[ks] = 0.21686275848693795
L1[ks] = 0.21635650396915546
L1[ks] = 0.21620001630624514
L1[ks] = 0.21590348239499058
L1[ks] = 0.21584936761175003
L1[ks] = 0.21584671614171042
L1[ks] = 0.21583647847005794
L1[ks] = 0.21582036042931124
L1[ks] = 0.2158261085086408


In [26]:
@show mean(w_arr[1:10]);
@show mean(w_arr[11:20]);
@show mean(w_arr[21:30]);
@show mean(w_arr[31:40]);
@show mean(w_arr[41:50]);
@show mean(w_arr[51:60]);
@show mean(w_arr[61:70]);
@show mean(w_arr[71:80]);
@show mean(w_arr[81:90]);
@show mean(w_arr[91:100]);
@show mean(w_arr[101:110]);
@show mean(w_arr[111:120]);
@show mean(w_arr[121:130]);
@show mean(w_arr[131:140]);
@show mean(w_arr[141:150]);
fields
wdf = DataFrame(w_arr)
CSV.write("w_arr.csv",wdf)
Xcsvdf = DataFrame(X);
names!(Xcsvdf, fields[1:15]);

mean(w_arr[1:10]) = [-0.169423, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mean(w_arr[11:20]) = [-0.12467, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.135113, 0.0, 0.0]
mean(w_arr[21:30]) = [-0.125115, 0.0, 0.0, 0.0, 0.0, -0.0354768, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.145129, 0.0, 0.0]
mean(w_arr[31:40]) = [-0.119286, 0.0, 0.0, 0.0, 0.0, -0.0377151, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0378203, 0.169176, 0.0, 0.0]
mean(w_arr[41:50]) = [-0.116251, 0.0, 0.0, 0.0, 0.0, -0.0273105, 0.00683902, 0.0, 0.0, 0.0, -0.064451, 0.0343964, 0.198416, 0.0, 0.0]
mean(w_arr[51:60]) = [-0.114245, 0.0, 0.0, 0.0, 0.0, -0.0271974, 0.011654, -0.0188557, 0.0, 0.0, -0.0602373, 0.0356362, 0.195324, 0.0, 0.0]
mean(w_arr[61:70]) = [-0.114261, 0.0, 0.0, 0.0, 0.0, -0.0194245, 0.0194245, -0.0188578, 0.0, 0.0, -0.0602025, 0.0356294, 0.195274, 0.0, 0.0]
mean(w_arr[71:80]) = [-0.114226, 0.0, 0.0, 0.000572434, 0.0, -0.0190079, 0.0190079, -0.0179498, 0.0, -0.0234897, -0.053333, 0.0358963, 0.197

In [72]:
loss = QuadLoss()
reg = ZeroReg()
w_final = proxgrad(loss,reg,X_train,Y_train[:])
yhat_train = X_train*w_final;

@show in_sample_l1 = mean(abs(Y_train - yhat_train))

yhat = X_test*w_final;

@show test_l1 = mean(abs(Y_test - yhat))

test_l1s = Y_test - yhat;
in_sample_l1s = Y_train - yhat_train;

@show test_l1*std_Y

@show std_Y

CSV.write("Xcsv",Xcsvdf)
CSV.write("testl1scsv",DataFrame(test_l1s))
CSV.write("insample_l1s",DataFrame(in_sample_l1s))
CSV.write("YCSV", DataFrame(Y))


@show w_final
median(Y*std_Y + mean_Y)


in_sample_l1 = mean(abs(Y_train - yhat_train)) = 0.24867411823467198
test_l1 = mean(abs(Y_test - yhat)) = 0.249961740331546
test_l1 * std_Y = 0.017404544996969487
std_Y = 0.0696288358925823
w_final = [9.54975e-5, 0.0172975, -0.0312056, 0.0544202, 0.019533, -0.0253295, 0.0253295, -0.0468568, -0.0299096, -0.0340446, -0.088703, 0.0881432, 0.383356, -0.0132815, 0.0716744]


0.0135